Copyright (c) 2020. Cognitive Scale Inc. All rights reserved.
Licensed under CognitiveScale Example Code [License](https://github.com/CognitiveScale/cortex-certifai-examples/blob/master/LICENSE.md)


# Alternate formulations for regression 

In this notebook we'll show how to modify the formulation of what constitutes a 'significant change' for the purposes of deciding how much change is required for an outcome to be considered different (counterfactual) to the original data point's prediction.

We'll base it on the primary [Regression example notebook](./Regression.ipynb) and use the same datasets, and models as that example does.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display
from sklearn.model_selection import train_test_split
import numpy as np
import random
import pprint

from sklearn.svm import SVR, LinearSVR
from sklearn.linear_model import Lasso
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import r2_score

from certifai.common.utils.encoding import CatEncoder
from certifai.scanner.builder import (CertifaiScanBuilder, CertifaiPredictorWrapper, CertifaiModel, CertifaiModelMetric,
                                      CertifaiDataset, CertifaiGroupingFeature, CertifaiDatasetSource,
                                      CertifaiPredictionTask, CertifaiTaskOutcomes, CertifaiOutcomeValue)
from certifai.scanner.report_utils import scores, construct_scores_dataframe
from certifai.scanner.explanation_utils import explanations, construct_explanations_dataframe, counterfactual_changes

In [2]:
# Prepare datasets for test/train split
base_path = '..'
all_data_file = f"{base_path}/datasets/auto_insurance_claims_dataset.csv"
explanation_data_file = f"{base_path}/datasets/auto_insurance_explan.csv"
RANDOM_SEED = 42

df = pd.read_csv(all_data_file)

cat_columns = [
    'State Code',
    'Coverage',
    'Education',
    'EmploymentStatus',
    'Gender',
    'Location Code',
    'Marital Status',
    'Policy',
    'Claim Reason',
    'Sales Channel',
    'Vehicle Class',
    'Vehicle Size',
]
label_column = "Total Claim Amount"


Y = df[label_column]
X = df.drop(label_column, axis=1)

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.20, random_state=RANDOM_SEED)

encoder = CatEncoder(cat_columns, X)

# Train models

def build_model(data, name, model_family, test=None):
    if test is None:
        test = data
        
    if model_family == 'SVM':
        parameters = {'C':[0.1, .5, 1, 2, 4, 10], 'epsilon':[0, 0.01, 0.1, 0.5, 1, 2, 4]}
        m = LinearSVR()
    elif model_family == 'Lasso':
        parameters = {'alpha': [0.001,0.01,.1]}
        m = Lasso()
    model = GridSearchCV(m, parameters, cv=3)
    model.fit(data[0], data[1])

    r2 = r2_score(test[1], model.predict(test[0]))
    print(f"{name} R-Squared: {r2}")
    return model

linl1_model = build_model((encoder(X_train.values), Y_train),
                          "LinL1",
                          "Lasso",
                          (encoder(X_test.values), Y_test))


svm_model = build_model((encoder(X_train.values), Y_train),
                          "SVM",
                          "SVM",
                          (encoder(X_test.values), Y_test))

Objective did not converge. You might want to increase the number of iterations. Duality gap: 6620064.389394514, tolerance: 41582.473948259176
Objective did not converge. You might want to increase the number of iterations. Duality gap: 6309746.68589443, tolerance: 41650.022503835404
Objective did not converge. You might want to increase the number of iterations. Duality gap: 11363590.2834258, tolerance: 41791.14840787425
Objective did not converge. You might want to increase the number of iterations. Duality gap: 738733.0027146786, tolerance: 41582.473948259176
Objective did not converge. You might want to increase the number of iterations. Duality gap: 599623.1800493002, tolerance: 41650.022503835404
Objective did not converge. You might want to increase the number of iterations. Duality gap: 591762.0417108089, tolerance: 41791.14840787425


LinL1 R-Squared: 0.763031334080441
SVM R-Squared: 0.7460085022147884


In [3]:
# Wrap the models for use by Certifai as a local model
linl1_model_proxy = CertifaiPredictorWrapper(linl1_model, encoder=encoder)
svm_model_proxy = CertifaiPredictorWrapper(svm_model, encoder=encoder)

In [4]:
# Make a scan object given the task definition (which encompasses the specifics of the forumlation desired)
def make_scan(task):
    # Create the scan object from scratch using the ScanBuilder class
    scan = CertifaiScanBuilder.create('regression_test_use_case',
                                      prediction_task=task)

    # Add our local models
    first_model = CertifaiModel('LinL1', local_predictor=linl1_model_proxy)
    scan.add_model(first_model)

    second_model = CertifaiModel('SVM', local_predictor=svm_model_proxy)
    scan.add_model(second_model)


    # Add datasets to the scan
    eval_dataset = CertifaiDataset('evaluation', CertifaiDatasetSource.csv(all_data_file))
    scan.add_dataset(eval_dataset)
    scan.evaluation_dataset_id = eval_dataset.id

    # For the sake of illustration we'll just extract explanations for a few examples
    NUM_EXPLANATIONS = 5
    explan_df = pd.read_csv(explanation_data_file)[:NUM_EXPLANATIONS]
    explan_dataset = CertifaiDataset('explanation', CertifaiDatasetSource.dataframe(explan_df))
    scan.add_dataset(explan_dataset)
    scan.explanation_dataset_id = explan_dataset.id

    # Because the datasets contain a ground truth outcome column which the model does not
    # expect to receive as input we need to state that in the dataset schema (since it cannot
    # be inferred from the CSV)
    scan.dataset_schema.outcome_feature_name = 'Total Claim Amount'

    # Setup an evaluation that just produces explanations in the interests of keeping this example
    # simpler
    scan.add_evaluation_type('explanation')
    
    return scan

# Set up absolute-threshold formulation

Here we'll define settlement values above \\$500 as being favorable.  That is - all predictions above \\$500
will now be considered favorable (an absolute fixed threshold) rather than those that are 0.5 standard deviations above the original prediction (which is what the baseline exaple did)

In [5]:
# Set the favorable direction to be increasing, and we'll consider
# predictions above the mean-prediction for
task = CertifaiPredictionTask(CertifaiTaskOutcomes.regression(True, absolute_threshold=500),
                              prediction_description='Amount of Settled Claim')

In [6]:
# Create the scan object for this formulation
scan = make_scan(task)

In [7]:
# Run the scan.
# By default this will write the results into individual report files (one per model and evaluation
# type) in the 'reports' directory relative to this notebook. This may be disabled by specifying
# `write_reports=False` as below
# The result is a dictionary of dictionaries of reports.  The top level dict key is the evaluation type
# and the second level key is model id.
results = scan.run(write_reports=False)

Starting scan with model_use_case_id: 'regression_test_use_case' and scan_id: '474f1312aef5'
[--------------------] 2020-12-21 12:33:09.918688 - 0 of 2 reports (0.0% complete) - Running explanation evaluation for model: LinL1
[##########----------] 2020-12-21 12:33:24.827183 - 1 of 2 reports (50.0% complete) - Running explanation evaluation for model: SVM
[####################] 2020-12-21 12:33:40.345202 - 2 of 2 reports (100.0% complete) - Completed all evaluations


In [8]:
# Using Certifai's explanation utilities we can programmatically explore counterfactuals produced
# during the explanation evaluation. Below we examine only a single explanation for each model
# by displaying the original input data followed by what features were changed by each
# counterfactual. Note, that for this regression use case a counterfactual will produced in the
# favorable direction (increasing) and in the unfavorable direction (decreasing)
linl1_explanations = construct_explanations_dataframe(explanations(results, model_id='LinL1'))
svm_explanations = construct_explanations_dataframe(explanations(results, model_id='SVM'))

def display_explanation(df):
    df_original = df[df['instance']=='original']
    df_original.style.hide_index()
    display(df_original)
    changes = counterfactual_changes(df)
    display(changes)
    if len(changes) < 2:
        print("No counterfactual found")

In [9]:
print("Explanations for LinL1 Model (settlements over $500 favorable):\n")
for idx in linl1_explanations['row'].unique():
    display_explanation(linl1_explanations[linl1_explanations['row'] == idx]) 

Explanations for LinL1 Model (settlements over $500 favorable):



,model,row,instance,cf_num,cf_type,prediction,fitness,State Code,Claim Amount,Coverage,...,Monthly Premium Auto,Months Since Last Claim,Months Since Policy Inception,Number of Open Complaints,Number of Policies,Policy,Claim Reason,Sales Channel,Vehicle Class,Vehicle Size
0,LinL1,1,original,0,original prediction,539.67,0.00,NE,387.36,Basic,...,105,18,50,0,1,Personal L3,Hail,Agent,Sports Car,Medsize


,instance,cf_num,cf_type,prediction,fitness,Monthly Premium Auto
0,original,0,original prediction,539.67,0.00,105
0,counterfactual,1,prediction decreased,499.10,2.25,97


,model,row,instance,cf_num,cf_type,prediction,fitness,State Code,Claim Amount,Coverage,...,Monthly Premium Auto,Months Since Last Claim,Months Since Policy Inception,Number of Open Complaints,Number of Policies,Policy,Claim Reason,Sales Channel,Vehicle Class,Vehicle Size
2,LinL1,2,original,0,original prediction,330.83,0.00,IA,1212.84,Basic,...,61,7,36,0,2,Personal L3,Hail,Agent,Four-Door Car,Medsize


,instance,cf_num,cf_type,prediction,fitness,Monthly Premium Auto
0,original,0,original prediction,330.83,0.00,61
0,counterfactual,1,prediction increased,503.27,0.53,95


...

,instance,cf_num,cf_type,prediction,fitness,Location Code,Monthly Premium Auto
0,original,0,original prediction,119.08,0.00,Rural,100
0,counterfactual,1,prediction increased,500.42,0.95,Suburban,101


,model,row,instance,cf_num,cf_type,prediction,fitness,State Code,Claim Amount,Coverage,...,Monthly Premium Auto,Months Since Last Claim,Months Since Policy Inception,Number of Open Complaints,Number of Policies,Policy,Claim Reason,Sales Channel,Vehicle Class,Vehicle Size
6,LinL1,4,original,0,original prediction,-22.16,0.00,MO,795.62,Basic,...,67,25,41,1,2,Personal L3,Collision,Web,Two-Door Car,Medsize


,instance,cf_num,cf_type,prediction,fitness,Location Code,Monthly Premium Auto
0,original,0,original prediction,-22.16,0.00,Rural,67
0,counterfactual,1,prediction increased,501.19,0.38,Suburban,96


,model,row,instance,cf_num,cf_type,prediction,fitness,State Code,Claim Amount,Coverage,...,Monthly Premium Auto,Months Since Last Claim,Months Since Policy Inception,Number of Open Complaints,Number of Policies,Policy,Claim Reason,Sales Channel,Vehicle Class,Vehicle Size
8,LinL1,5,original,0,original prediction,537.89,0.00,NE,234.76,Basic,...,69,14,10,1,1,Personal L3,Collision,Branch,Four-Door Car,Medsize


,instance,cf_num,cf_type,prediction,fitness,Location Code
0,original,0,original prediction,537.89,0.00,Suburban
0,counterfactual,1,prediction decreased,381.45,1.00,Urban


In [10]:
print("Explanations for SVM Model (settlements over $500 favorable):\n")
for idx in svm_explanations['row'].unique():
    display_explanation(svm_explanations[svm_explanations['row'] == idx]) 

Explanations for SVM Model (settlements over $500 favorable):



,model,row,instance,cf_num,cf_type,prediction,fitness,State Code,Claim Amount,Coverage,...,Monthly Premium Auto,Months Since Last Claim,Months Since Policy Inception,Number of Open Complaints,Number of Policies,Policy,Claim Reason,Sales Channel,Vehicle Class,Vehicle Size
0,SVM,1,original,0,original prediction,501.80,0.00,NE,387.36,Basic,...,105,18,50,0,1,Personal L3,Hail,Agent,Sports Car,Medsize


,instance,cf_num,cf_type,prediction,fitness,Monthly Premium Auto
0,original,0,original prediction,501.80,0.00,105
0,counterfactual,1,prediction decreased,496.98,18.00,104


,model,row,instance,cf_num,cf_type,prediction,fitness,State Code,Claim Amount,Coverage,...,Monthly Premium Auto,Months Since Last Claim,Months Since Policy Inception,Number of Open Complaints,Number of Policies,Policy,Claim Reason,Sales Channel,Vehicle Class,Vehicle Size
2,SVM,2,original,0,original prediction,300.27,0.00,IA,1212.84,Basic,...,61,7,36,0,2,Personal L3,Hail,Agent,Four-Door Car,Medsize


,instance,cf_num,cf_type,prediction,fitness,Monthly Premium Auto
0,original,0,original prediction,300.27,0.00,61
0,counterfactual,1,prediction increased,502.59,0.43,103


...

,instance,cf_num,cf_type,prediction,fitness,Income,Location Code,Monthly Premium Auto
0,original,0,original prediction,161.77,0.00,63834,Rural,100
0,counterfactual,1,prediction increased,500.03,0.76,62945,Suburban,105


,model,row,instance,cf_num,cf_type,prediction,fitness,State Code,Claim Amount,Coverage,...,Monthly Premium Auto,Months Since Last Claim,Months Since Policy Inception,Number of Open Complaints,Number of Policies,Policy,Claim Reason,Sales Channel,Vehicle Class,Vehicle Size
6,SVM,4,original,0,original prediction,18.26,0.00,MO,795.62,Basic,...,67,25,41,1,2,Personal L3,Collision,Web,Two-Door Car,Medsize


,instance,cf_num,cf_type,prediction,fitness,Location Code,Monthly Premium Auto
0,original,0,original prediction,18.26,0.00,Rural,67
0,counterfactual,1,prediction increased,500.97,0.34,Suburban,102


,model,row,instance,cf_num,cf_type,prediction,fitness,State Code,Claim Amount,Coverage,...,Monthly Premium Auto,Months Since Last Claim,Months Since Policy Inception,Number of Open Complaints,Number of Policies,Policy,Claim Reason,Sales Channel,Vehicle Class,Vehicle Size
8,SVM,5,original,0,original prediction,488.51,0.00,NE,234.76,Basic,...,69,14,10,1,1,Personal L3,Collision,Branch,Four-Door Car,Medsize


,instance,cf_num,cf_type,prediction,fitness,Monthly Premium Auto
0,original,0,original prediction,488.51,0.00,69
0,counterfactual,1,prediction increased,502.96,6.00,72


# Set up percentile threshold formulation

Similarly we can define a fixed threshold in terms of percentiles of the evaluation dataset's predicted values.
In this case let's define 'favorable' as the top 20% of the distribution.

In [11]:
# Set the favorable direction to be increasing, and we'll consider
# predictions above the mean-prediction for
task = CertifaiPredictionTask(CertifaiTaskOutcomes.regression(True, absolute_percentile=80),
                              prediction_description='Amount of Settled Claim')

In [12]:
# Create the scan object for this formulation
scan = make_scan(task)

In [13]:
results = scan.run(write_reports=False)

linl1_explanations = construct_explanations_dataframe(explanations(results, model_id='LinL1'))
svm_explanations = construct_explanations_dataframe(explanations(results, model_id='SVM'))

[--------------------] 2020-12-21 12:33:41.069183 - 0 of 2 reports (0.0% complete) - Starting scan with model_use_case_id: 'regression_test_use_case' and scan_id: '156144ea443e'
[--------------------] 2020-12-21 12:33:41.069420 - 0 of 2 reports (0.0% complete) - Running explanation evaluation for model: LinL1
[##########----------] 2020-12-21 12:33:56.321381 - 1 of 2 reports (50.0% complete) - Running explanation evaluation for model: SVM
[####################] 2020-12-21 12:34:10.998020 - 2 of 2 reports (100.0% complete) - Completed all evaluations


In [14]:
print("Explanations for LinL1 Model (80th percentile favorable):\n")
for idx in linl1_explanations['row'].unique():
    display_explanation(linl1_explanations[linl1_explanations['row'] == idx]) 

Explanations for LinL1 Model (80th percentile favorable):



,model,row,instance,cf_num,cf_type,prediction,fitness,State Code,Claim Amount,Coverage,...,Monthly Premium Auto,Months Since Last Claim,Months Since Policy Inception,Number of Open Complaints,Number of Policies,Policy,Claim Reason,Sales Channel,Vehicle Class,Vehicle Size
0,LinL1,1,original,0,original prediction,539.67,0.00,NE,387.36,Basic,...,105,18,50,0,1,Personal L3,Hail,Agent,Sports Car,Medsize


,instance,cf_num,cf_type,prediction,fitness,Monthly Premium Auto
0,original,0,original prediction,539.67,0.00,105
0,counterfactual,1,prediction increased,610.67,1.29,119


,model,row,instance,cf_num,cf_type,prediction,fitness,State Code,Claim Amount,Coverage,...,Monthly Premium Auto,Months Since Last Claim,Months Since Policy Inception,Number of Open Complaints,Number of Policies,Policy,Claim Reason,Sales Channel,Vehicle Class,Vehicle Size
2,LinL1,2,original,0,original prediction,330.83,0.00,IA,1212.84,Basic,...,61,7,36,0,2,Personal L3,Hail,Agent,Four-Door Car,Medsize


,instance,cf_num,cf_type,prediction,fitness,Monthly Premium Auto
0,original,0,original prediction,330.83,0.00,61
0,counterfactual,1,prediction increased,609.77,0.33,116


...

,instance,cf_num,cf_type,prediction,fitness,Location Code,Monthly Premium Auto
0,original,0,original prediction,119.08,0.00,Rural,100
0,counterfactual,1,prediction increased,612.00,0.44,Suburban,123


,model,row,instance,cf_num,cf_type,prediction,fitness,State Code,Claim Amount,Coverage,...,Monthly Premium Auto,Months Since Last Claim,Months Since Policy Inception,Number of Open Complaints,Number of Policies,Policy,Claim Reason,Sales Channel,Vehicle Class,Vehicle Size
6,LinL1,4,original,0,original prediction,-22.16,0.00,MO,795.62,Basic,...,67,25,41,1,2,Personal L3,Collision,Web,Two-Door Car,Medsize


,instance,cf_num,cf_type,prediction,fitness,Location Code,Monthly Premium Auto
0,original,0,original prediction,-22.16,0.00,Rural,67
0,counterfactual,1,prediction increased,607.69,0.26,Suburban,117


,model,row,instance,cf_num,cf_type,prediction,fitness,State Code,Claim Amount,Coverage,...,Monthly Premium Auto,Months Since Last Claim,Months Since Policy Inception,Number of Open Complaints,Number of Policies,Policy,Claim Reason,Sales Channel,Vehicle Class,Vehicle Size
8,LinL1,5,original,0,original prediction,537.89,0.00,NE,234.76,Basic,...,69,14,10,1,1,Personal L3,Collision,Branch,Four-Door Car,Medsize


,instance,cf_num,cf_type,prediction,fitness,Monthly Premium Auto
0,original,0,original prediction,537.89,0.00,69
0,counterfactual,1,prediction increased,608.90,1.29,83


In [15]:
print("Explanations for SVM Model (80th percentile favorable):\n")
for idx in svm_explanations['row'].unique():
    display_explanation(svm_explanations[svm_explanations['row'] == idx]) 

Explanations for SVM Model (80th percentile favorable):



,model,row,instance,cf_num,cf_type,prediction,fitness,State Code,Claim Amount,Coverage,...,Monthly Premium Auto,Months Since Last Claim,Months Since Policy Inception,Number of Open Complaints,Number of Policies,Policy,Claim Reason,Sales Channel,Vehicle Class,Vehicle Size
0,SVM,1,original,0,original prediction,501.80,0.00,NE,387.36,Basic,...,105,18,50,0,1,Personal L3,Hail,Agent,Sports Car,Medsize


,instance,cf_num,cf_type,prediction,fitness,Monthly Premium Auto
0,original,0,original prediction,501.80,0.00,105
0,counterfactual,1,prediction increased,564.42,1.38,118


,model,row,instance,cf_num,cf_type,prediction,fitness,State Code,Claim Amount,Coverage,...,Monthly Premium Auto,Months Since Last Claim,Months Since Policy Inception,Number of Open Complaints,Number of Policies,Policy,Claim Reason,Sales Channel,Vehicle Class,Vehicle Size
2,SVM,2,original,0,original prediction,300.27,0.00,IA,1212.84,Basic,...,61,7,36,0,2,Personal L3,Hail,Agent,Four-Door Car,Medsize


,instance,cf_num,cf_type,prediction,fitness,Monthly Premium Auto
0,original,0,original prediction,300.27,0.00,61
0,counterfactual,1,prediction increased,565.21,0.33,116


...

,instance,cf_num,cf_type,prediction,fitness,Location Code,Monthly Premium Auto
0,original,0,original prediction,161.77,0.00,Rural,100
0,counterfactual,1,prediction increased,562.59,0.50,Suburban,118


,model,row,instance,cf_num,cf_type,prediction,fitness,State Code,Claim Amount,Coverage,...,Monthly Premium Auto,Months Since Last Claim,Months Since Policy Inception,Number of Open Complaints,Number of Policies,Policy,Claim Reason,Sales Channel,Vehicle Class,Vehicle Size
6,SVM,4,original,0,original prediction,18.26,0.00,MO,795.62,Basic,...,67,25,41,1,2,Personal L3,Collision,Web,Two-Door Car,Medsize


,instance,cf_num,cf_type,prediction,fitness,Location Code,Monthly Premium Auto
0,original,0,original prediction,18.26,0.00,Rural,67
0,counterfactual,1,prediction increased,563.60,0.27,Suburban,115


,model,row,instance,cf_num,cf_type,prediction,fitness,State Code,Claim Amount,Coverage,...,Monthly Premium Auto,Months Since Last Claim,Months Since Policy Inception,Number of Open Complaints,Number of Policies,Policy,Claim Reason,Sales Channel,Vehicle Class,Vehicle Size
8,SVM,5,original,0,original prediction,488.51,0.00,NE,234.76,Basic,...,69,14,10,1,1,Personal L3,Collision,Branch,Four-Door Car,Medsize


,instance,cf_num,cf_type,prediction,fitness,Monthly Premium Auto
0,original,0,original prediction,488.51,0.00,69
0,counterfactual,1,prediction increased,560.77,1.20,84


# Notes

It is apparent from the last formulation that using a percentile of the outcome distribution is *not* the
same as specifying an a-priori fixed value as the favorability threshold.  This is apparent by comparing the results of the two models which show the SVM model had an 80% percentile at about \\$560, whereas the logistic model had one at a little over \\$600.  It is also noteable that the logistic model predicts some negative values, which are clearly not realistic.  Both of these are indications that the logistic model (which will have a linear decision surface) is insufficiently complex to capture the data behaviour at the extremes.  This is despite its R-squared actually being better than that of the SVM.

For comparing different models one would typically therefore prefer to specify a fixed threshold informed by domain knowledge rather than an outcome percentile, though an outcome percentile formulation can be useful especically during initial exploration of a use-case.